In [45]:
import tensorflow as tf
import numpy as np
import time
from datetime import timedelta

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/MNIST/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [7]:
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)
num_channels = 1
num_classes = 10

# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

In [34]:
tf.reset_default_graph()
def conv_layer(name, input, filter_size, in_filter, out_filter):
    with tf.variable_scope(name):
        weights = tf.get_variable(name='dw', shape=[filter_size, filter_size, in_filter, out_filter], initializer=tf.truncated_normal_initializer(stddev=0.05))
        bias = tf.get_variable(name='bw', shape=[out_filter], initializer=tf.constant_initializer(0.05))
        layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')
        layer += bias
        layer = tf.nn.relu(layer)
        layer = tf.nn.max_pool(value=layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        return layer

def flat_layer(layer):
    layer_shape = layer.get_shape()
    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

def full_conn(input, input_dim, out_dim):
    weights = tf.get_variable(name='dw', shape=[input_dim, out_dim], initializer=tf.truncated_normal_initializer(stddev=0.05))
    bias = tf.get_variable('bw', shape=[out_dim], initializer=tf.constant_initializer(0.05))
    return tf.nn.xw_plus_b(input, weights, bias)

In [35]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [36]:
layer_conv1 = conv_layer('conv1', input=x_image, filter_size=filter_size1, in_filter=num_channels, out_filter=num_filters1)
layer_conv2 = conv_layer('conv2', input=layer_conv1, filter_size=filter_size2, in_filter=num_filters1, out_filter=num_filters2)
layer_flat, num_features = flat_layer(layer_conv2)
layer_fc = full_conn(input=layer_flat, input_dim=num_features, out_dim=num_classes)

In [37]:
y_pred = tf.nn.softmax(layer_fc)
y_pred_cls = tf.argmax(y_pred, dimension=1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc, labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [38]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [39]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [43]:
train_batch_size = 64
total_iterations = 0

def optimize(num_iterations):
    # Ensure we update the global variable rather than a local copy.
    global total_iterations

    # Start-time used for printing time-usage below.
    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):
        x_batch, y_true_batch = data.train.next_batch(train_batch_size)
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        session.run(optimizer, feed_dict=feed_dict_train)
        if i % 100 == 0:
            # Calculate the accuracy on the training-set.
            acc = session.run(accuracy, feed_dict=feed_dict_train)
            # Message for printing.
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
            print(msg.format(i + 1, acc))
    total_iterations += num_iterations
    
    end_time = time.time()
    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [47]:
optimize(num_iterations=99)

Optimization Iteration:    101, Training Accuracy:  62.5%
Time usage: 0:00:14


In [3]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2,3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3,2], name='b')
c = tf.matmul(a, b)

In [103]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [5]:
sess.run(c)

array([[ 22.,  28.],
       [ 49.,  64.]], dtype=float32)

In [10]:
sess.run(tf.slice(a, [1, 0], [1, 1]))

array([[ 4.]], dtype=float32)

In [11]:
e = tf.constant([1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6], shape=[3,2,3], name='e')

In [23]:
sess.run(tf.slice(e, [1,1,1], [2,1,2]))

array([[[4, 4]],

       [[6, 6]]], dtype=int32)

In [31]:
sess.run(tf.strided_slice(e, [1,1,1], [3,2,3], [2,1,1]))

array([[[4, 4]]], dtype=int32)

In [113]:
a = tf.constant([0.01,3, 0.01], name='a')
b = tf.constant([0.0,1.0,0.0],name='b')
c = tf.nn.softmax_cross_entropy_with_logits(logits=a, labels=b)

In [107]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), 0)

In [112]:
softmax([0.01, 3, 0.01])

array([ 0.04569197,  0.90861606,  0.04569197])

In [114]:
sess.run(c)

0.095832743

In [2]:
m = np.array([0.11,0.1,1,2,2,2,3,3,3,4,4,4])
n = np.array([1,2,5,1,2,5,1,2,5,1,2,5])

In [63]:
t = np.reshape(m, [2,2,3])
f = np.reshape(n, [12,1])

In [12]:
ids1 = np.argwhere(m == 2).ravel()
ids2 = np.argwhere(m == 3).ravel()
np.concatenate((ids1, ids2))

array([3, 4, 5, 6, 7, 8])

In [75]:
i = np.argwhere(m==0.11).ravel()

In [91]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [119]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        m = tf.get_variable('dt1', [1])

In [109]:
m.name

'foo/dt:0'

In [123]:
with tf.variable_scope("foo", reuse=True):
    with tf.variable_scope("bar"):
        m2 = tf.get_variable("dt1", [1])

In [149]:
x = tf.placeholder(tf.float32, shape=[None, 5], name='x')
x1 = tf.placeholder(tf.float32, shape=[None, 5], name='x1')

In [150]:
x_batch = np.array([[1,2,3,4,5], [5,4,3,2,1]])

In [151]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(x, feed_dict={x:x_batch})
print(x.get_shape())

array([[ 1.,  2.,  3.,  4.,  5.],
       [ 5.,  4.,  3.,  2.,  1.]], dtype=float32)

In [153]:
print(x.get_shape()[1])

5


In [17]:
arr = np.arange(9)
np.random.shuffle(arr)
arr.reshape((3,3))

array([[6, 1, 5],
       [4, 3, 8],
       [0, 2, 7]])

In [18]:
np.ones(4)*25

array([ 25.,  25.,  25.,  25.])

In [20]:
np.sum(np.array([12,21]))

33

In [26]:
np.unique(np.array([1,1,22]))

array([ 1, 22])

In [31]:
np.count_nonzero(m==3)


3

In [35]:
np.concatenate([np.array([]),[1,2,3]])

array([ 1.,  2.,  3.])

In [146]:
int(3.3)

3

In [147]:
c = [("a",2,3), ("2",1,4)]
np.random.choice([4,5,3,2], 0)

array([], dtype=int64)

In [70]:
import re
re.split(r'[\s+,]', "I like play , and you")

['I', 'like', 'play', '', '', 'and', 'you']

In [75]:
np.argwhere(m[[1,2,3]] > 1).ravel()

array([2])

In [86]:
m.reshape([3,2,2])

array([[[ 0.11,  0.1 ],
        [ 1.  ,  2.  ]],

       [[ 2.  ,  2.  ],
        [ 3.  ,  3.  ]],

       [[ 3.  ,  4.  ],
        [ 4.  ,  4.  ]]])

In [87]:
t = m.reshape([3,2,2])

In [88]:
np.max(t, 2)

array([[ 0.11,  2.  ],
       [ 2.  ,  3.  ],
       [ 4.  ,  4.  ]])

In [99]:
t[(2,0),(1,1)]

array([[ 4.,  4.],
       [ 1.,  2.]])

In [116]:
np.argsort(m)

array([ 1,  0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [118]:
m

array([ 0.11,  0.1 ,  1.  ,  2.  ,  2.  ,  2.  ,  3.  ,  3.  ,  3.  ,
        4.  ,  4.  ,  4.  ])

In [119]:
set(m)- set([1,2,3])

{0.10000000000000001, 0.11, 4.0}

In [125]:
m[list(set([1,2,3]))]

array([ 0.1,  1. ,  2. ])

In [124]:
np.array(list(set([1,2,3])))

array([1, 2, 3])

In [136]:
m.reshape([3,4])

array([[ 0.11,  0.1 ,  1.  ,  2.  ],
       [ 2.  ,  2.  ,  3.  ,  3.  ],
       [ 3.  ,  4.  ,  4.  ,  4.  ]])

In [137]:
j = np.argsort(m.reshape([3,4]), axis=0).astype(np.int32)

In [164]:
j

array([[0, 0, 0, 0],
       [1, 1, 1, 1],
       [2, 2, 2, 2]], dtype=int32)

In [158]:
22 in [list(range(1,23)), list(range(1,23))]

False

In [163]:
2 in list(range(10, 40)) + list(range(50, 80)) + list(range(90, 120))

False

In [178]:
np.argsort(m.reshape([3,4]), axis=1)

array([[1, 0, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])

In [179]:
np.abs(m)

array([ 0.11,  0.1 ,  1.  ,  2.  ,  2.  ,  2.  ,  3.  ,  3.  ,  3.  ,
        4.  ,  4.  ,  4.  ])